In [1]:
import os
import sys
import numpy as np
from collections import defaultdict
import gmpy2
import copy
import logging

inputfilepath = 'C:\EMOLGUI\A_O_C\Day16.txt'
#inputfilepath = 'C:\EMOLGUI\A_O_C\Day16.sample.txt'

In [2]:
with open(inputfilepath,'r') as f:
    valveList = {}
    count = 0
    for line in f:
        valve = {}
        valve['open'] = False
        valve['index'] = count
        count += 1
        valve_blocks = line.rstrip().split(';')
        valve_name = valve_blocks[0].split(' has ')[0].split()[1]
        flow_rate = valve_blocks[0].split(' has ')[1].split('=')[1]
        valve['flow_rate'] = flow_rate
        next_valves = valve_blocks[1].split(' valve')[1].strip('s').strip().split(', ')
        valve['next_valves'] = next_valves
        valveList[valve_name] = valve
        print(f'{valve_name} has flow rate {flow_rate}. It leads to {next_valves}.')
print(valveList)


AA has flow rate 0. It leads to ['RZ', 'QQ', 'FH', 'IM', 'VJ'].
FE has flow rate 0. It leads to ['TM', 'TR'].
QZ has flow rate 19. It leads to ['HH', 'OY'].
TU has flow rate 17. It leads to ['NJ', 'IN', 'WN'].
RG has flow rate 0. It leads to ['IK', 'SZ'].
TM has flow rate 0. It leads to ['FE', 'JH'].
JH has flow rate 4. It leads to ['NW', 'QQ', 'TM', 'VH', 'AZ'].
NW has flow rate 0. It leads to ['JH', 'OB'].
BZ has flow rate 0. It leads to ['XG', 'XF'].
VS has flow rate 0. It leads to ['FF', 'GC'].
OI has flow rate 20. It leads to ['SY'].
IK has flow rate 0. It leads to ['RG', 'TR'].
RO has flow rate 0. It leads to ['UZ', 'YL'].
LQ has flow rate 0. It leads to ['IZ', 'PA'].
GG has flow rate 18. It leads to ['GH', 'VI'].
NJ has flow rate 0. It leads to ['TU', 'UZ'].
SY has flow rate 0. It leads to ['OI', 'ZL'].
HH has flow rate 0. It leads to ['QZ', 'WN'].
RZ has flow rate 0. It leads to ['AA', 'UZ'].
OF has flow rate 0. It leads to ['YL', 'IZ'].
IZ has flow rate 9. It leads to ['OF', '

In [3]:
timeLimit = 30
startingValve = 0

In [4]:
distanceMatrix = np.zeros((len(valveList),len(valveList)))
flowsList = np.zeros(len(valveList))
for valveName, valveInfo in valveList.items():
    origId = valveInfo['index']
    flowsList[origId] = valveInfo['flow_rate']
    distanceMatrix[origId,origId] = -1
    for nextValve in valveInfo['next_valves']:        
        nextId = valveList[nextValve]['index']
        distanceMatrix[origId,nextId] += 1

round = 2
while np.size(np.where(distanceMatrix == 0)) > 0:
    print(distanceMatrix)
    for row in range(len(valveList)):
        print(f'Checking origin valve {row}')
        for firstNeigh in range(len(valveList)):
            distanceToFirstNb = distanceMatrix[row,firstNeigh]
            if distanceToFirstNb <= 0:
                continue
            #print(f'    next valve {firstNeigh}')
            for finalNeighbor in range(len(valveList)):
                if finalNeighbor == row:
                    continue
                relayDistance = distanceMatrix[firstNeigh,finalNeighbor]
                if relayDistance <= 0:
                    continue
                totalDistance = distanceToFirstNb + relayDistance
                if not totalDistance == round:
                    #print(f'        {totalDistance} needs to match {round}')
                    continue
                if distanceMatrix[row,finalNeighbor] == 0:
                    distanceMatrix[row,finalNeighbor] = totalDistance
                else:
                    distanceMatrix[row,finalNeighbor] = min(totalDistance, distanceMatrix[row,finalNeighbor])

    round += 1

distanceMatrix, np.size(np.where(distanceMatrix == 0)), flowsList

[[-1.  0.  0. ...  0.  0.  0.]
 [ 0. -1.  0. ...  0.  0.  0.]
 [ 0.  0. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ...  0. -1.  0.]
 [ 0.  0.  0. ...  0.  0. -1.]]
Checking origin valve 0
Checking origin valve 1
Checking origin valve 2
Checking origin valve 3
Checking origin valve 4
Checking origin valve 5
Checking origin valve 6
Checking origin valve 7
Checking origin valve 8
Checking origin valve 9
Checking origin valve 10
Checking origin valve 11
Checking origin valve 12
Checking origin valve 13
Checking origin valve 14
Checking origin valve 15
Checking origin valve 16
Checking origin valve 17
Checking origin valve 18
Checking origin valve 19
Checking origin valve 20
Checking origin valve 21
Checking origin valve 22
Checking origin valve 23
Checking origin valve 24
Checking origin valve 25
Checking origin valve 26
Checking origin valve 27
Checking origin valve 28
Checking origin valve 29
Checking origin valve 30
Checking origin valve 31
Checking origin va

(array([[-1.,  4.,  7., ...,  5.,  5.,  5.],
        [ 4., -1., 10., ...,  7.,  8.,  3.],
        [ 7., 10., -1., ...,  3.,  2.,  8.],
        ...,
        [ 5.,  7.,  3., ..., -1.,  4.,  8.],
        [ 5.,  8.,  2., ...,  4., -1.,  6.],
        [ 5.,  3.,  8., ...,  8.,  6., -1.]]),
 0,
 array([ 0.,  0., 19., 17.,  0.,  0.,  4.,  0.,  0.,  0., 20.,  0.,  0.,
         0., 18.,  0.,  0.,  0.,  0.,  0.,  9.,  0.,  0., 11.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  5., 21.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  0., 10., 16.,  0.,
         0., 22.,  0., 13.]))

In [5]:
def nextMove(valveList,currentValve,closedValves,distanceMatrix,flowsList,timeLimit,valueSoFar):
    if (timeLimit < 2) or (len(closedValves) == len(valveList)):
        #print(f'    Completed path {closedValves} with value {valueSoFar}')
        return 0, []
    bestValue = 0
    bestNextValve = -1
    bestFollowingValves = []
    for nextValve in range(len(valveList)):
        if (nextValve == currentValve):
            #print(f'{nextValve} is the same as {currentValve}')
            continue
        if (nextValve in closedValves):
            #print(f'{nextValve} is closed {closedValves}')
            continue
        distance = distanceMatrix[currentValve,nextValve]
        if (distance >= (timeLimit)):
            #print(f'Distance {distance} is too high for remaining time {timeLimit}')
            continue
        openValue = flowsList[nextValve]
        if(openValue == 0):
            #print(f'Vale {openValue} is zero')
            continue

        #print(f'Adding Value from valve {currentValve} to valve {nextValve}. Distance is {distance}, remaining time is {timeLimit}. Path so far {closedValves}')
        
        nexttimeLimit = timeLimit - (distance+1) #Move and open the valve
        value = openValue * nexttimeLimit
        nextValue, followingValves = nextMove(valveList,nextValve,closedValves + [nextValve], distanceMatrix,flowsList, nexttimeLimit, valueSoFar + value)
        totalValue = value + nextValue
        if totalValue > bestValue:
            bestValue = totalValue
            bestNextValve = nextValve
            bestFollowingValves = [nextValve] + followingValves
        if False:
            print(f'    Value so far {valueSoFar + bestValue} in path {closedValves + [nextValve]}')
    return bestValue, bestFollowingValves
        

In [141]:
        
value, sequence = nextMove(valveList,startingValve,[0],distanceMatrix,flowsList,30,0)

In [142]:
value, sequence

(1641.0, [20, 53, 2, 3, 34, 55, 14, 23])

#### QUESTION 2

In [6]:
# This actually works, though I do not think it should
value, sequence = nextMove(valveList,startingValve,[0],distanceMatrix,flowsList,26,0)
valueElf, sequenceElf = nextMove(valveList,startingValve,sequence,distanceMatrix,flowsList,26,0)

In [8]:
value, valueElf, sequence, sequenceElf, value + valueElf

(1209.0, 1052.0, [20, 53, 2, 3, 34, 32, 50], [47, 55, 14, 23, 10, 49], 2261.0)

In [158]:
def nextMoveWithElephant(valveList,currentValveList,closedValves,distanceMatrix,flowsList,timeLimitList,valueSoFar):
    #if (timeLimitList < 2) or (len(closedValves) == len(valveList)):
        #print(f'    Completed path {closedValves} with value {valueSoFar}')
    #    return 0, []
    bestValue = 0
    bestNextValve = -1
    bestFollowingValves = []
    nextValvesList = copy.copy(currentValveList)
    nextTimeLimitList = copy.copy(timeLimitList)

    bestValueList = [0, 0]
    for index, currentValve in enumerate(currentValveList):
        timeLimit = timeLimitList[index]
        nextValves = copy.copy(currentValveList)
        #if timeLimit < max(timeLimitList):
        #    continue

        for nextValve in range(len(valveList)):
            #if (nextValve in currentValveList):
                #print(f'{nextValve} is the same as {currentValve}')
                #continue
            if (nextValve in closedValves):
                #print(f'{nextValve} is closed {closedValves}')
                continue
            if (nextValve in nextValvesList):
                continue
            distance = distanceMatrix[currentValve,nextValve]
            if (distance >= timeLimit):
                #print(f'Distance {distance} is too high for remaining time {timeLimit}')
                continue
            nextValves[index] = nextValve
            openValue = flowsList[nextValve]
            if(openValue == 0):
                #print(f'Vale {openValue} is zero')
                continue
            
            #print(f'Adding Value from valve {currentValve} to valve {nextValve}. Distance is {distance}, remaining time is {timeLimit}. Path so far {closedValves}')
        
            nextTimeLimitList[index] = timeLimit - (distance+1) #Move and open the valve
            value = openValue * nextTimeLimitList[index]
            nextValue, followingValves = nextMoveWithElephant(valveList,nextValvesList,closedValves + [nextValve], distanceMatrix,flowsList, nextTimeLimitList, valueSoFar + value)
            totalValue = value + nextValue

            if totalValue > bestValue:
                bestValue = totalValue
                bestNextValve = nextValve
                bestFollowingValves = [nextValve] + followingValves
            if False:
                print(f'    Value so far {valueSoFar + bestValue} in path {closedValves + [nextValve]}')
        if bestValue > 0:
            return bestValue, bestFollowingValves
    return 0, []

        

In [176]:
timeLimit = 17
value, sequence = nextMoveWithElephant(valveList,[startingValve, startingValve],[0],distanceMatrix,flowsList,[timeLimit, timeLimit],0)

In [177]:
value, sequence

(925.0, [20, 34, 50, 3, 35, 53, 32, 2])